In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days


In [2]:
import skimage.io
from scipy.ndimage.filters import convolve

#note: this requires the starter code for the assignments!
from common.plotting import plot_mat

In [3]:
import theano
import theano.tensor.signal.downsample
import lasagne
import time,colorsys
from skimage.transform import rotate
print theano.config.device

gpu


Using gpu device 0: GeForce GTX 780 (CNMeM is enabled)


In [4]:
from IPython.display import SVG
def svgdotprint(g):
    return SVG(theano.printing.pydotprint(g, return_image=True, format='svg'))

In [5]:
sys.stdout,TMP = open("lab110-01-U7.txt","w"),sys.stdout

In [6]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten, Mapping
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}))

cifar_train = CIFAR10(("train",), subset=slice(None,40000))
#this stream will shuffle the MNIST set and return us batches of 100 examples
cifar_train_stream = DataStream.default_stream(
    cifar_train,
    iteration_scheme=ShuffledScheme(cifar_train.num_examples, 25))
                                               
cifar_validation = CIFAR10(("train",), subset=slice(40000, None))

# We will use larger portions for testing and validation
# as these dont do a backward pass and reauire less RAM.
cifar_validation_stream = DataStream.default_stream(
    cifar_validation, iteration_scheme=SequentialScheme(cifar_validation.num_examples, 25))
cifar_test = CIFAR10(("test",))
cifar_test_stream = DataStream.default_stream(
    cifar_test, iteration_scheme=SequentialScheme(cifar_test.num_examples, 25))

In [7]:
print "The streams return batches containing %s" % (cifar_train_stream.sources,)

print "Each trainin batch consits of a tuple containing:"
for element in next(cifar_train_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)
    
print "Validation/test batches consits of tuples containing:"
for element in next(cifar_test_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)

In [8]:
#based on https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/merge.py
class RandomChoseLayer(lasagne.layers.ElemwiseMergeLayer):
    def __init__(self, incomings, cropping=None, **kwargs):
        super(RandomChoseLayer, self).__init__(incomings, theano.tensor.add, **kwargs)

    def get_output_for(self, inputs, **kwargs):
        self._srng = RandomStreams(lasagne.random.get_rng().randint(1, 2147462579))
        x = self._srng.uniform(size=(1,), low=0.0, high=len(inputs))
        L = [(i <= x)*(x < i+1)  for i in range(len(inputs))]
        inputs = [input * coeff if coeff != 1 else input
                  for coeff, input in zip(L, inputs)]

        # pass scaled inputs to the super class for summing
        return super(RandomChoseLayer, self).get_output_for(inputs, **kwargs)

In [9]:
def RandomSliceLayer(input,size):
    L = []
    a,b = lasagne.layers.get_output_shape(input)[-2:]
    for x in range(0,a-size):
        l1 = lasagne.layers.SliceLayer(input, indices=slice(x,x+size), axis=-1)
        for y in range(0,b-size):
            l2 = lasagne.layers.SliceLayer(l1, indices=slice(y,y+size), axis=-2)
            L.append(l2)
    return RandomChoseLayer(L)
    

In [10]:
def build_deepCNiN(input_var=None, l = 5, k = 100, drop = [0,0,0.1,0.2,0.3,0.4,0.5]):

    network = lasagne.layers.InputLayer(shape=(None, 3, 96, 96),
                                        input_var=input_var)

    network = lasagne.layers.Conv2DLayer(
            lasagne.layers.dropout(network, p=drop[0]),
            num_filters=k, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    
    for i in range(l):
        network = lasagne.layers.MaxPool2DLayer(network,pool_size=(2,2))
        network = lasagne.layers.NINLayer(network,num_units=k*(i+1))
        network = lasagne.layers.Conv2DLayer(
            lasagne.layers.dropout(network, p=drop[i+1]),
            num_filters=k*(i+2), filter_size=(2, 2),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    network = lasagne.layers.NINLayer(network,num_units=k*(l+1))
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=drop[-1]),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [11]:
def RGBtoYUV(R,G,B):
    Y = 0.299 * R + 0.587 * G  + 0.114 * B
    U = 0.492 * (B - Y)
    V = 0.877 * (R - Y)
    return Y,U,V

def model_change(picture,function=colorsys.rgb_to_hsv):
    picture2 = zeros(shape = shape(picture))
    for i in range(shape(picture)[1]):
        for j in range(shape(picture)[2]):
            a,b,c = picture[:,i,j]
            a,b,c = function((a+1)/2,(b+1)/2,(c+1)/2)
            picture2[:,i,j] = a*2-1,b*2-1,c*2-1
    return picture2

def DataTransform(inputs,outputs,num=1):
    #return inputs,outputs
    a,b = (96,96)
    inp = []
    out = []
    
    for x,y in zip(inputs,outputs):
        for i in range(num):
            _,sa,sb = shape(x)
            rx,ry = random.randint(-16,16)+sa,random.randint(-16,16)+sb
            xx = zeros(shape=(3,a,b),dtype = float32)
            xx[:,rx:rx+sa,ry:ry+sb] = x
            ang = random.randint(-15,15)
            for i in range(3):
                xx[i] = rotate(xx[i], ang)
            inp.append(xx)
            out.append(y)
            #inp.append(model_change(xx,function=RGBtoYUV))
            #out.append(y)
    return inp,out


for inputs, targets in cifar_train_stream.get_epoch_iterator():
    A,B = DataTransform(inputs,targets)
    break
    for S in A:
        aa,bb,cc = shape(S)
        S = S.reshape((1,aa,bb,cc))
        plot_mat(S.transpose(1,0,2,3), cmap='gray')
        show()
    break

In [12]:
try:
    print 'Nets:', len(list_of_nets)
except:
    print 'new list'
    list_of_nets = []

In [ ]:
def snapshot(net):
    return lasagne.layers.get_all_param_values(net)
def restore(net, params):
    lasagne.layers.set_all_param_values(net,params)

In [ ]:
input_var = theano.tensor.tensor4('inputs')
target_var = theano.tensor.ivector('targets')

net = build_deepCNiN(input_var)

prediction = lasagne.layers.get_output(net)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
l2_penalty = lasagne.regularization.regularize_layer_params(lasagne.layers.get_all_layers(net), lasagne.regularization.l2)
weight_penalty = theano.tensor.scalar('weight_penalty')
loss = loss.mean()#+weight_penalty*l2_penalty

l_rate = theano.tensor.scalar('l_rate')
params = lasagne.layers.get_all_params(net, trainable=True)
updates = lasagne.updates.nesterov_momentum(
    loss, params, learning_rate=l_rate, momentum=0.75)
test_prediction = lasagne.layers.get_output(net, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = theano.tensor.mean(theano.tensor.eq(theano.tensor.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var,l_rate], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
num_epochs = 51
i = 0
weight_penalty_value = 0
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for inputs, targets in cifar_train_stream.get_epoch_iterator():
        in4,ou4 = DataTransform(inputs, targets[:,0],2)
        i += 1
        K = 10000
        lrate = 3e-2 * K / np.maximum(K, i)
        train_err += train_fn(in4, ou4,lrate)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for inputs, targets in cifar_validation_stream.get_epoch_iterator():
        in4,ou4 = DataTransform(inputs, targets[:,0])
        err, acc = val_fn(in4, ou4)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    if epoch % 10 == 0:
        test_err = 0
        test_acc = 0
        test_batches = 0
        for inputs, targets in cifar_train_stream.get_epoch_iterator():
            in4,ou4 = DataTransform(inputs, targets[:,0])
            err, acc = val_fn(in4, ou4)
            test_err += err
            test_acc += acc
            test_batches += 1
        print("Final results:")
        print("  train loss:\t\t\t{:.6f}".format(test_err / test_batches))
        print("  train accuracy:\t\t{:.2f} %".format(
                test_acc / test_batches * 100))
        if test_acc/test_batches - val_acc/val_batches > 0.09 and weight_penalty_value < 0.005:
            weight_penalty_value += 0.0005
        print 'weight_penalty_value:', weight_penalty_value
    sys.stdout.flush()

test_err = 0
test_acc = 0
test_batches = 0
for inputs, targets in cifar_train_stream.get_epoch_iterator():
    in4,ou4 = DataTransform(inputs, targets[:,0])
    err, acc = val_fn(in4, ou4)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  train loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  train accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))    
# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for inputs, targets in cifar_validation_stream.get_epoch_iterator():
    in4,ou4 = DataTransform(inputs, targets[:,0])
    err, acc = val_fn(in4, ou4)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))
list_of_nets.append((test_acc / test_batches * 100,snapshot(net)))

In [ ]:
TMP,sys.stdout = sys.stdout,TMP
TMP.close()

In [23]:
pred_fn = theano.function([input_var], argmax(prediction,axis=1))

In [27]:
test_result = []
for inputs, targets in cifar_test_stream.get_epoch_iterator():
    in4,ou4 = DataTransform(inputs, targets[:,0])
    pre = pred_fn(in4)
    test_result += zip(pre,ou4)

In [37]:
st = open('result_NiN_100','w')
for (x,y) in test_result:
    st.write('%d %d\n'%(x,y))
st.close()